In [1]:
%pylab inline
import numpy as np
import scipy as sp
import tensorflow as tf
import pandas as pd
import seaborn as sns
import itertools,time
import sys, os
from collections import OrderedDict
from copy import deepcopy
import numpy as np
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import itertools
import pickle
import time

Populating the interactive namespace from numpy and matplotlib


/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
dataset_tr = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/NTP/process_UCI/M_nips.full_docs.mat'
data_tr = sp.io.loadmat(dataset_tr)['M'].T.toarray()

vocab_ = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/NTP/process_UCI/vocab.nips.txt'
vocab_ = open(vocab_,'r').readlines()
vocab=np.array([word for word in vocab_])

print data_tr.shape, vocab.shape

(1500, 12419) (12419,)


In [3]:
vocab_size=len(vocab)

In [4]:
n_samples_tr = data_tr.shape[0]
docs_tr = data_tr

In [5]:
def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)
def log_dir_init(fan_in, fan_out,topics=50): 
    return tf.log((1.0/topics)*tf.ones([fan_in, fan_out]))

tf.reset_default_graph()
class VariationalAutoencoder(object):
    """ 
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """
    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus, 
                 learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        print 'Learning Rate:', self.learning_rate
        
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        self.keep_prob = tf.placeholder(tf.float32)
        self.is_training = tf.placeholder(tf.bool, [], name='is_training')
        self.samples = tf.placeholder(tf.int32)
        
        self.h_dim = float(network_architecture["n_z"])
        self.a0 = np.ones((1 , 2)).astype(np.float32)/2
        self.a1 = np.ones((1 , self.h_dim)).astype(np.float32)/self.h_dim
        self.a2 = np.ones((1 , self.h_dim)).astype(np.float32)/self.h_dim
        
        self.mu2a0 = tf.constant((np.log(self.a0).T-np.mean(np.log(self.a0),1)).T)
        self.var2a0 = tf.constant(  ( ( (1.0/self.a0)*( 1 - (2.0/2.0) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a0,1) ).T  )
        
        self.mu2a1 = tf.constant((np.log(self.a1).T-np.mean(np.log(self.a1),1)).T)
        self.var2a1 = tf.constant(  ( ( (1.0/self.a1)*( 1 - (2.0/self.h_dim) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a1,1) ).T  )
        
        self.mu2a2 = tf.constant((np.log(self.a2).T-np.mean(np.log(self.a2),1)).T)
        self.var2a2 = tf.constant(  ( ( (1.0/self.a2)*( 1 - (2.0/self.h_dim) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a2,1) ).T  )
        
        
        
        # Create autoencoder network
        self._create_network()
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        self._create_loss_optimizer()
        
        # Initializing the tensor flow variables
        init = tf.initialize_all_variables()

        # Launch the session
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
    
    def _create_network(self):
        n_z = self.network_architecture["n_z"]
        # Initialize autoencode network weights and biases
        self.network_weights = self._initialize_weights(**self.network_architecture)
        '''Adding 2 subtopics and 1 super topic.'''
        self.z_mean0,self.z_log_sigma_sq0 = \
            self._recognition_network(self.network_weights["weights_recog"], 
                                      self.network_weights["biases_recog"])
            
        self.z_mean_,self.z_log_sigma_sq_ = \
            self._recognition_network(self.network_weights["sub_01"], 
                                      self.network_weights["biases_sub_01"])
            
        self.z_mean1=self.z_mean_[:,:2]
        self.z_mean2=self.z_mean_[:,2:]
        self.z_log_sigma_sq1=self.z_log_sigma_sq_[:,:2]
        self.z_log_sigma_sq2=self.z_log_sigma_sq_[:,2:]
        
            
        self.z_mean__,self.z_log_sigma_sq__ = \
            self._recognition_network(self.network_weights["sub_02"], 
                                      self.network_weights["biases_sub_02"])
            
        self.z_mean3=self.z_mean__[:,:n_z]
        self.z_mean4=self.z_mean__[:,n_z:]
        self.z_log_sigma_sq3=self.z_log_sigma_sq__[:,:n_z]
        self.z_log_sigma_sq4=self.z_log_sigma_sq__[:,n_z:]
        


        # Draw one sample z from Gaussian distribution
        '''Adding another noise variable for super toipc: hard coding atm'''
        eps0 = tf.random_normal((self.samples, 2), 0, 1, 
                               dtype=tf.float32)
        
        eps1 = tf.random_normal((self.samples, 2), 0, 1, 
                               dtype=tf.float32)
        eps2 = tf.random_normal((self.samples, n_z), 0, 1, 
                               dtype=tf.float32)
        '''Adding RT for subtopics'''
        self.z0 = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(self.z_mean0, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq0)), eps0))))
        self.z0 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z0,
                                                             is_training=self.is_training,updates_collections=None))
        
        self.sigma0 = tf.exp(self.z_log_sigma_sq0)
        
        self.z1 = tf.add(self.z_mean1, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq1)), eps1))
        self.z1 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z1,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma1 = tf.exp(self.z_log_sigma_sq1)
        
        self.z2 = tf.add(self.z_mean2, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq2)), eps1))
        self.z2 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z2,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma2 = tf.exp(self.z_log_sigma_sq2)
        
        self.z3 = tf.add(self.z_mean3, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq3)), eps2))
        self.z3 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z3,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma3 = tf.exp(self.z_log_sigma_sq3)
        
        self.z4 = tf.add(self.z_mean4, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq4)), eps2))
        self.z4 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z4,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma4 = tf.exp(self.z_log_sigma_sq4)
        
        '''Adding subtopic reconstruction'''
            
#         self.theta = tf.matmul(self.z0,tf.concat(0,[self.z1, self.z2]))
        
        self.theta_ = tf.squeeze(tf.map_fn(lambda i: 
                                          tf.matmul(tf.expand_dims(tf.gather(self.z0,i),0),
                                                    tf.concat(0,[tf.expand_dims(tf.gather(self.z1,i),0),
                                                                tf.expand_dims(tf.gather(self.z2,i),0)])),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
        
        self.theta = tf.squeeze(tf.map_fn(lambda i: 
                                          tf.matmul(tf.expand_dims(tf.gather(self.theta_,i),0),
                                                    tf.concat(0,[tf.expand_dims(tf.gather(self.z3,i),0),
                                                                tf.expand_dims(tf.gather(self.z4,i),0)])),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
            
        self.x_reconstr_mean = \
            self._generator_network(tf.nn.dropout(self.theta, self.keep_prob),
                                    self.network_weights["weights_gener"],
                                    self.network_weights["biases_gener"])
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2, 
                            n_hidden_gener_1,  
                            n_input, n_z):
        all_weights = dict()
        with tf.variable_scope("super"):
            all_weights['weights_recog'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, 2]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, 2])
            }
            all_weights['biases_recog'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([2], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([2], dtype=tf.float32))
            }
        with tf.variable_scope("sub01"):
            all_weights['sub_01'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, 2*2]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, 2*2])}
            all_weights['biases_sub_01'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([2*2], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([2*2], dtype=tf.float32))
            }
        with tf.variable_scope("sub02"):
            all_weights['sub_02'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, n_z*2]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, n_z*2])
            }
            all_weights['biases_sub_02'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([n_z*2], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([n_z*2], dtype=tf.float32))
            }
        with tf.variable_scope("gen"):
            all_weights['weights_gener'] = {
                'h2': tf.Variable(xavier_init(n_z, n_hidden_gener_1))
            }
            all_weights['biases_gener'] = {
                'b2': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32))
            }
        return all_weights
            
    def _recognition_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network)
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        layer_do = tf.nn.dropout(layer_2, self.keep_prob)
        
        z_mean = tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_mean']),
                        biases['out_mean']),is_training=self.is_training,updates_collections=None)
        z_log_sigma_sq = \
            tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma']), 
                   biases['out_log_sigma']),is_training=self.is_training,updates_collections=None)     
        
        return (z_mean, z_log_sigma_sq)

    def _generator_network(self,z, weights, biases):
        # Generate probabilistic decoder (decoder network)
#         self.layer_do_0 = tf.nn.dropout(tf.nn.softmax(tf.contrib.layers.batch_norm(z)), self.keep_prob)
        self.layer_do_0 =z
        x_reconstr_mean = tf.add(tf.matmul(self.layer_do_0, 
                                           tf.nn.softmax(tf.contrib.layers.batch_norm(weights['h2'],\
is_training=self.is_training,updates_collections=None))),0.0)
#         x_reconstr_mean = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(
#                     tf.matmul(self.layer_do_0, weights['h2']),0.0)))
    
        return x_reconstr_mean

            
    def _create_loss_optimizer(self):
        
        self.x_reconstr_mean+=1e-10
     
        reconstr_loss = \
            -tf.reduce_sum(self.x * tf.log(self.x_reconstr_mean),1)#/tf.reduce_sum(self.x,1) 

    
        latent_loss0 = 0.5*( tf.reduce_sum(tf.div(self.sigma0,self.var2a0),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a0 - self.z_mean0),self.var2a0),
                  (self.mu2a0 - self.z_mean0)),1) - 2 +\
                           tf.reduce_sum(tf.log(self.var2a0),1)  - tf.reduce_sum(self.z_log_sigma_sq0  ,1) ) 
        
        latent_loss1 = 0.5*( tf.reduce_sum(tf.div(self.sigma1,self.var2a0),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a0 - self.z_mean1),self.var2a0),
                  (self.mu2a0 - self.z_mean1)),1) - 2 +\
                           tf.reduce_sum(tf.log(self.var2a0),1)  - tf.reduce_sum(self.z_log_sigma_sq1  ,1) ) 
        
        latent_loss2 = 0.5*( tf.reduce_sum(tf.div(self.sigma2,self.var2a0),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a0 - self.z_mean2),self.var2a0),
                  (self.mu2a0 - self.z_mean2)),1) - 2 +\
                           tf.reduce_sum(tf.log(self.var2a0),1)  - tf.reduce_sum(self.z_log_sigma_sq2  ,1) ) 
        
        latent_loss3 = 0.5*( tf.reduce_sum(tf.div(self.sigma3,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z_mean3),self.var2a1),
                  (self.mu2a1 - self.z_mean3)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z_log_sigma_sq3  ,1) )
        
        latent_loss4 = 0.5*( tf.reduce_sum(tf.div(self.sigma4,self.var2a2),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a2 - self.z_mean4),self.var2a2),
                  (self.mu2a2 - self.z_mean4)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a2),1)  - tf.reduce_sum(self.z_log_sigma_sq4  ,1) )

        
        
#         latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq 
#                                            - tf.square(self.z_mean) 
#                                            - tf.exp(self.z_log_sigma_sq), 1)


        self.cost = tf.reduce_mean(reconstr_loss) \
    + tf.reduce_mean(latent_loss0 + latent_loss1 + latent_loss2 + latent_loss3 + latent_loss4) # average over batch
        
        # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate,beta1=0.9).minimize(self.cost)
        
    def partial_fit(self, X):
        """Train model based on mini-batch of input data.
        
        Return cost of mini-batch.
        """
#         opt, cost,emb,kld = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2'],
#                                           self.kld),feed_dict={self.x: X,self.keep_prob: .9})
        opt,cost,emb = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2']
                                      ),feed_dict={self.x: X,self.keep_prob: .7,self.is_training:True,self.samples:1})
        return cost,emb
    
    def test(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost,e1,e2,e3,e4 = self.sess.run((self.cost,self.z1,self.z2,self.z3,self.z4),
                                        feed_dict={self.x: np.expand_dims(X, axis=0),self.keep_prob: 1.0,
                                                   self.is_training:False,self.samples:1})
        return cost,e1,e2,e3,e4
    
    def batchtest(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost,e1,e2,e3,e4 = self.sess.run((self.cost,self.z1,self.z2,self.z3,self.z4),
                                        feed_dict={self.x: X,self.keep_prob: 1.0,
                                                   self.is_training:False,self.samples:1})
        return cost,e1,e2,e3,e4


In [8]:
tf.reset_default_graph()
cost_plot=[]
test_batch = create_minibatch(docs_tr.astype('float32'))
def train(network_architecture, learning_rate=0.0001,
          batch_size=100, training_epochs=10, display_step=5):
    vae = VariationalAutoencoder(network_architecture, 
                                 learning_rate=learning_rate, 
                                 batch_size=batch_size)
    emb=0
    avg_kld=0
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        avg_kld=0.
        total_batch = int(n_samples_tr / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = minibatches.next()
            # Fit training using batch data
            cost,emb = vae.partial_fit(batch_xs)
#             c,a,b,k,l = vae.batchtest(batch_xs)

#             sys.exit()
            # Compute average loss
            avg_cost += cost / n_samples_tr * batch_size
#             avg_kld += c / n_samples_tr * batch_size
            
            if np.isnan(avg_cost):
                print epoch,i,np.sum(batch_xs,1).astype(np.int),batch_xs.shape
                return vae,emb
#                 sys.exit()
        
        # Display logs per epoch step
        if epoch % display_step == 0:
            cost_plot.append(avg_cost)
            print "Epoch:", '%04d' % (epoch+1), \
                  "cost=", "{:.9f}".format(avg_cost)
#             print 'avg_cost = ',avg_kld
            test_c=0.
#             for i in range(int(n_samples_te / batch_size)):
#                 c,a,b,k,l = vae.batchtest(test_batch.next())
#                 test_c+=c / n_samples_te * batch_size
#             print 'avg_test_cost = ',test_c
    return vae,emb

In [9]:
tf.reset_default_graph()
network_architecture = \
    dict(n_hidden_recog_1=400, # 1st layer encoder neurons
         n_hidden_recog_2=400, # 2nd layer encoder neurons
         n_hidden_gener_1=docs_tr.shape[1], # 1st layer decoder neurons
         n_input=docs_tr.shape[1], # MNIST data input (img shape: 28*28)
         n_z=50)  # dimensionality of latent space

batch_size=200
learning_rate=0.005
def create_minibatch(data):
    rng = np.random.RandomState(10)
    
    while True:
        # Return random data samples of a size 'minibatch_size' at each iteration
        ixs = rng.randint(data.shape[0], size=batch_size)
        yield data[ixs]
start=time.time()
minibatches = create_minibatch(docs_tr.astype('float32'))
vae,emb = train(network_architecture, training_epochs=500,batch_size=batch_size,learning_rate=learning_rate)
# plt.plot(cost_plot)
print 'it took',str(time.time()-start),'seconds'


'''
Change log: decoder has no dropout or additional tranformations
'''

/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)
/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Learning Rate: 0.005
Epoch: 0001 cost= 15710.180338542
Epoch: 0006 cost= 13740.960286458
Epoch: 0011 cost= 12995.959375000
Epoch: 0016 cost= 12722.149348958
Epoch: 0021 cost= 12525.194270833
Epoch: 0026 cost= 12197.385156250
Epoch: 0031 cost= 12114.079687500
Epoch: 0036 cost= 12047.057812500
Epoch: 0041 cost= 11832.433203125
Epoch: 0046 cost= 11864.229296875
Epoch: 0051 cost= 11742.731640625
Epoch: 0056 cost= 11573.179687500
Epoch: 0061 cost= 11535.747395833
Epoch: 0066 cost= 11468.102473958
Epoch: 0071 cost= 11439.629427083
Epoch: 0076 cost= 11530.482291667
Epoch: 0081 cost= 11448.210677083
Epoch: 0086 cost= 11378.112760417
Epoch: 0091 cost= 11268.363411458
Epoch: 0096 cost= 11238.934114583
Epoch: 0101 cost= 11306.197786458
Epoch: 0106 cost= 11344.061979167
Epoch: 0111 cost= 11207.384244792
Epoch: 0116 cost= 11243.599479167
Epoch: 0121 cost= 11137.137109375
Epoch: 0126 cost= 11148.302213542
Epoch: 0131 cost= 11176.147526042
Epoch: 0136 cost= 11119.773958333
Epoch: 0141 cost= 11114.797

'\nChange log: decoder has no dropout or additional tranformations\n'

In [10]:
def print_top_words(beta, feature_names, n_top_words=5):
    for i in range(len(beta)):
        print(" ".join([feature_names[j]
            for j in beta[i].argsort()[:-n_top_words - 1:-1]]))

print_top_words(emb, "".join(vocab).split('\n'))

conscious item psychological psychology consciousness
codeword code decoded ziv encoding
policy reinforcement tsitsiklis transition action
nectar bees encounter montague foraging
frequency cell signal response zee
rfwr expertise pert blending expert
parcor speech jaw tongue koike
distribution matrix gaussian mean non
location field image motion visual
layer input unit paperr network
differ mainly common tend greater
hassibi order error thorbergsson gradient
layer hinton network input interactively
neuron synaptic synapses potential spike
linesearch kushner annealing algorithm ribiere
intrapersonal scientist perona feret rois
gopal sdh program marko nlist
activity field cell visual neuron
string recurrent languages automata subgraph
john kevin pierre scott alan
graph clustering cluster gurewitz musk
positive examples auer class algorithm
hopf gutfreund hopfield dynamic phase
pacemaker beer chiel heterogeneity raven
bound theorem positive function cornell
control dynamic controller robot

In [13]:

cost=[]
e1=[]
e2=[]
e3=[]
e4=[]
idx=0
for doc in docs_te:
    idx+=1
    doc = doc.astype('float32')
    n_d = np.sum(doc)
    c,a,b,k,l=vae.test(doc)
    cost.append(c)
#     print c,n_d
    e1.append(a)
    e2.append(b)
    e3.append(k)
    e4.append(l)
    if idx%1000==0:
        print idx
    
print ((np.mean(np.array(cost))))
# print np.mean(e1,0)
# print np.mean(e2,0)

1000
2000
3000
4000
5000
6000
7000
3708.81


In [16]:
print np.argsort(np.mean(e1,0)[0])+1
print np.argsort(np.mean(e2,0)[0])+1
print np.argsort(np.mean(e3,0)[0])+1
print np.argsort(np.mean(e4,0)[0])+1
print np.mean(e1,0)#[0]
print np.mean(e2,0)#[0]
print np.sort(np.mean(e3,0))#[0]
print np.sort(np.mean(e4,0))#[0]
print np.exp(np.mean(np.array(cost)))

[2 1]
[1 2]
[ 2 36 49 13 22  7 37 14 24 46 32 44 16 26 48 18 23 25  9  1 50  8 40 20 12
  6 31 10 28 11 15 27 29 34 42 38 33 21 41 45 39  3  5  4 30 19 47 17 43 35]
[38 13 14 20 49 43  6 44 34 50 18 25 32 36 22  3 24  7 40 23 46 12  8 10 16
 27  9 28  1 26 48 19  4 45  5 42 11  2 31 35 29 47 41 17 21 30 37 15 33 39]
[[  1.00000000e+00   1.04386508e-07]]
[[  1.46643613e-22   1.00000000e+00]]
[[ 0.00334829  0.00536268  0.0056372   0.00964715  0.01178652  0.01187422
   0.01188467  0.01232468  0.01255479  0.01256892  0.01261481  0.01279969
   0.01294623  0.01468423  0.01486358  0.01507311  0.01508066  0.01690379
   0.01752479  0.01775655  0.01783933  0.01795567  0.01801715  0.01852449
   0.01868502  0.01873625  0.01886442  0.01917117  0.01997501  0.02026113
   0.02043691  0.02197537  0.02254776  0.0235034   0.02368179  0.02381792
   0.02418585  0.02497809  0.02586782  0.02708452  0.02931921  0.02949175
   0.02979021  0.02984348  0.03071339  0.03142248  0.03168741  0.03430611
   0.03759667 

In [19]:
pickle.dump(cost,open('cost','w'))
pickle.dump(e1,open('e1','w'))
pickle.dump(e2,open('e2','w'))

In [19]:
e=e2
print np.amax(np.mean(e,0))
print np.min(np.mean(e,0))

0.066614
0.00266949


In [98]:
test_batch = create_minibatch(docs_te.astype('float32'))
def test(vae,minibatches):
    emb=0
    avg_kld=0
    print batch_size
    # Training cycle
    for epoch in range(10):
        avg_cost = 0.
        avg_kld = 0.
        total_batch = int(n_samples_te / batch_size)
        print total_batch
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = minibatches.next()
            # Fit training using batch data
            cost,a,b = vae.batchtest(batch_xs)
            cost=cost
            # Compute average loss
            avg_cost += cost / total_batch#n_samples_tr * batch_size


        # Display logs per epoch step
#         if epoch % display_step == 0:
        cost_plot.append(avg_cost)
        print "Epoch:", '%04d' % (epoch+1), \
              "cost=", "{:.9f}".format(np.exp((avg_cost/90)))
        print 'avg_cost = ',avg_cost

test(vae,test_batch)

200
37
Epoch: 0001 cost= 1951.299849720
avg_cost =  681.862591718
37
Epoch: 0002 cost= 1802.893371419
avg_cost =  674.743327373
37
Epoch: 0003 cost= 2080.210038171
avg_cost =  687.620173274
37
Epoch: 0004 cost= 2131.480287028
avg_cost =  689.811478898
37
Epoch: 0005 cost= 1697.812007509
avg_cost =  669.3386082
37
Epoch: 0006 cost= 1812.600706940
avg_cost =  675.226615287
37
Epoch: 0007 cost= 2147.505452401
avg_cost =  690.485597353
37
Epoch: 0008 cost= 1754.365650309
avg_cost =  672.287635597
37
Epoch: 0009 cost= 1986.632824159
avg_cost =  683.477679278
37
Epoch: 0010 cost= 1301.936128341
avg_cost =  645.444698849


In [65]:
print np.mean(np.sum(docs_tr.astype('float32'),1))

88.0701


In [ ]:
[2 1 0 7 3 8 4 5 6 9]
thanks hus advance appreciate modem
article write like get one
god people bible faith one
team rangers game player leafs
people population arab village government
god people christian truth evidence
thanks motherboard sale hd ide
virtual software interface graphic application
government arab israeli turks population
spacecraft encryption key km flight

[4 1 9 2 8 6 5 0 3 7]
thanks motherboard sale hd ide
god people christian truth evidence
virtual software interface graphic application
god people bible faith one
article write like get one
people population arab village government
spacecraft encryption key km flight
thanks hus advance appreciate modem
government arab israeli turks population
team rangers game player leafs